In [ ]:
!pip install pandas

In [ ]:
# Leer CSV

import pandas as pd

df = pd.read_csv("preguntas_modelos_arquitecturas_gemm3.csv")
print(df)

In [ ]:
# Cargando instancias necesarias
import os
import sys
sys.path.append(os.path.abspath(".."))  # Ajusta según sea necesario

import gradio as gr
from pymongo import MongoClient
from rag.llm import LLM
from clinical_data.patients import PatientRepository
from clinical_data.clinical_history import ClinicalHistoryRepository
from dotenv import load_dotenv

load_dotenv()

# Crear el cliente
client = MongoClient(os.environ.get("DATABASE_CONNECTION"))

# Crear el repositorio
patient_repository = PatientRepository(client)
clinical_history_repository = ClinicalHistoryRepository(client)

# Crear conexión con LLM (Ollama)
llm = LLM()


In [ ]:
import time 

history = []
respouestas = []
tiempos_ejecucion = []

for index, row in df.iterrows():
    
    if row["pregunta"].lower() == "hola": # Estrategia para detectar cuando cambiar de usuario
        history = []

        patient = patient_repository.get_patient_by_id(str(row["identificacion_paciente"]))

        clinical_history = clinical_history_repository.get_clinical_history(
            patient_id=str(patient["_id"]))

        patient = {key: patient[key] for key in [
            "nombre_completo", "fecha_nacimiento", "estado_civil", "lugar_residencia"] if key in patient}

        summary = "\n\n".join(
            f"Fecha y hora de la atencion: {c['fecha_hora']}\nResumen de la atencion medica: {c['resumen']}"
            for c in clinical_history["atenciones"]
        )

        print("----------------------")

    print("USER: " + row["pregunta"])
    # Agregar mensaje normal del usuario
    history.append({"role": "user", "content": row["pregunta"]})

    inicio = time.time()
    if row["arquitectura"] == "contextual_rag":
        response = llm.get_response_contextual_rag(str(clinical_history["_id"]), patient, summary, history, row["modelo"])
    elif row["arquitectura"] == "simple_rag":
        response = llm.get_response_simple_rag(str(clinical_history["_id"]), patient, summary, history, row["modelo"])
    fin = time.time()

    print("RESPONSE: " + response)
    history.append({
        "role": "assistant",
        "content": response
    })
    respouestas.append(response)
    tiempos_ejecucion.append(fin-inicio)

df["respuesta"] = respouestas
df["tiempo_ejecucion"] = tiempos_ejecucion

In [ ]:
df